In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig

model_name = "../qwen2.5-3b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    output_hidden_states=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

#model.eval()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.0,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)

model.print_trainable_parameters()


trainable params: 1,843,200 || all params: 3,087,781,888 || trainable%: 0.0597


In [3]:
model.train()

prompt = "请用一句话解释什么是强化学习。"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
print(inputs)

for i in range(2) :
    outputs = model(**inputs)
    logits = outputs.logits  # [B, T, V]
    print("inputs:", inputs)
    print("output:", outputs.logits.shape)
    
    print("input_text:", tokenizer.decode(inputs["input_ids"][0], skip_special_tokens=True))
    text = tokenizer.decode(outputs.logits.argmax(dim=-1)[0])
    print("output_text:", text)

    # 取最后一个 token 的 logits
    last_logits = logits[:, -1, :]

    # 假设我们想“鼓励生成 token id = 42”
    target_token_id = torch.tensor([42], device=logits.device)

    with torch.no_grad():
        old_prob = torch.softmax(last_logits, dim=-1)[0, target_token_id]
        print("target token prob before update:", old_prob.item())

    loss = -torch.log_softmax(last_logits, dim=-1)[0, target_token_id]

    loss.backward()

    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=1e-4
    )

    optimizer.step()
    optimizer.zero_grad()

with torch.no_grad():
    outputs_new = model(**inputs)
    new_logits = outputs_new.logits[:, -1, :]
    new_prob = torch.softmax(new_logits, dim=-1)[0, target_token_id]
print("target token prob after update:", new_prob.item())

{'input_ids': tensor([[ 14880,  11622, 105321, 104136, 106582, 103980, 100134,   1773]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}
inputs: {'input_ids': tensor([[ 14880,  11622, 105321, 104136, 106582, 103980, 100134,   1773]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}
output: torch.Size([1, 8, 151936])
input_text: 请用一句话解释什么是强化学习。
output_text: 地中文描述什么是人工智能学习。
 �
target token prob before update: 5.960464477539063e-08
inputs: {'input_ids': tensor([[ 14880,  11622, 105321, 104136, 106582, 103980, 100134,   1773]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}
output: torch.Size([1, 8, 151936])
input_text: 请用一句话解释什么是强化学习。
output_text: 地中文描述什么是人工智能学习。
 �
target token prob before update: 5.960464477539063e-08
target token prob after update: 1.1920928955078125e-07


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchviz import make_dot
import copy

import numpy as np

def check_tensor_dtype(tensor, name):
    if tensor is None:
        print(f"{name}: is None")
        return
    print(f"{name}: dtype = {tensor.dtype}, device = {tensor.device}")

def reward_model(prompt_response) :
    reward = torch.tensor([1.0], dtype=torch.float16).to('cuda')
    reward = reward.clamp(-5, 5)
    return reward

class ValueHead(nn.Module) :
    def __init__(self) :
        super().__init__()
        self.fc = nn.LazyLinear(1).to(torch.float16)

    def forward(self, x) :
        return self.fc(x)

class Agent() :
    def __init__(self, model, gamma, epsilon, lam, epochs, device) :
        self.model = model
        self.value_head = ValueHead().to(device)
        self.optimizer = torch.optim.AdamW(
            list(self.model.parameters()) + list(self.value_head.parameters()),
            lr=1e-4
        )
        
        self.loss = nn.MSELoss().to(torch.float16)

        self.gamma = gamma
        self.epsilon = epsilon
        self.epochs = epochs
        self.lam = lam

        self.device = device

        self.total_loss = []
        
    def take_action(self, state) :
        output = self.model(**state, output_hidden_states=True)
        return output

    def update(self, transition_dict, inputs, response, prompt_response) :  
        for _ in range(self.epochs) :
            output = self.model(**prompt_response, output_hidden_states=True)
        
            T = len(transition_dict['action'])
            logits = output.logits[:,-T:,:]
            dist = torch.distributions.Categorical(logits=logits)
        
            action = torch.cat(transition_dict['action'], dim=0).to(self.device)
            log_p_new = dist.log_prob(action).sum()
            with torch.no_grad():
                log_p_old = torch.stack(transition_dict['log_p']).sum()
        
            ratio = torch.exp(log_p_new - log_p_old)
        
            reward = reward_model(response)
            adv = (reward - transition_dict['value_head'][-1]).detach()
            adv = (adv - adv.mean()) / (adv.std() + 1e-8)

            entropy = dist.entropy().sum()
        
            policy_loss = -torch.min(
                ratio * adv,
                torch.clamp(ratio, 1-self.epsilon, 1+self.epsilon) * adv
            ) - 0.01 * entropy

            new_value = self.value_head(
                output.hidden_states[-1][:, -1, :]
            )
            value_loss = F.mse_loss(new_value.squeeze(), reward.squeeze())
        
            loss = policy_loss + value_loss

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
        print(loss.item())

# agent init
gamma = 0.98
epsilon = 0.2
epochs = 10
lam = 1
agent = Agent(model, gamma, epsilon, lam, epochs, 'cuda')

return_list = []
epoch_length_list = []

epoch_num = 10

prompt = "请用一句话解释什么是强化学习。"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
newline_token_id = tokenizer.encode("<|endoftext|>", add_special_tokens=False)[0]

for i in range(epoch_num // 2) :
    for j in range(2) :
        observation = inputs
        done = False
        total_reward = 0
        iter = 0
        
        transition_dict = {'action': [], 'log_p':[], 'value_head':[]}
        while (not done and iter < 100):
            output = agent.take_action(observation)
            dist = torch.distributions.Categorical(logits=output.logits[:,-1,:])
            action = dist.sample()
            log_p = dist.log_prob(action)
            value_head = agent.value_head(output.hidden_states[-1][:,-1,:])

            next_input_ids = torch.cat([observation["input_ids"], action.unsqueeze(-1)], dim=1)
            next_attention_mask = torch.cat([observation["attention_mask"], torch.ones_like(action.unsqueeze(-1))], dim = 1)
            next_observation = {
                "input_ids": next_input_ids,
                "attention_mask": next_attention_mask
            }
            
            text = tokenizer.decode(next_input_ids[0])
            #print("output_text:", text)
            
            #print(action, observation["input_ids"], observation["attention_mask"], next_input_ids, next_attention_mask)
            
            done = (action == newline_token_id)
        
            # 采样的数据只是样本，用来计算优势，衡量和新策略的距离，都是标量，不用反向传播。
            transition_dict['log_p'].append(log_p.detach())
            transition_dict['action'].append(action.detach())
            transition_dict['value_head'].append(value_head.detach())

            if done :
                #print(next_observation['input_ids'], torch.cat(transition_dict['action'], dim=0))
                response_ids = next_input_ids[:,inputs['input_ids'].shape[1]:]
                response_attention_mask = next_attention_mask[:, inputs['attention_mask'].shape[1]:]
                response = {
                    "input_ids": response_ids,
                    "attention_mask": response_attention_mask
                }
                agent.update(transition_dict, inputs, response, next_observation)
            observation = next_observation
            iter += 1

/tmp/ipykernel_371324/4205823548.py:69: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /pytorch/aten/src/ATen/native/ReduceOps.cpp:1857.)
  adv = (adv - adv.mean()) / (adv.std() + 1e-8)


nan


/pytorch/aten/src/ATen/native/cuda/TensorCompare.cu:112: _assert_async_cuda_kernel: block: [0,0,0], thread: [0,0,0] Assertion `probability tensor contains either `inf`, `nan` or element < 0` failed.


AcceleratorError: CUDA error: device-side assert triggered
Search for `cudaErrorAssert' in https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__TYPES.html for more information.
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
